In [6]:
from gensim.models import doc2vec

import re
import pandas as pd
import numpy as np
import copy

# nltk processing
import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews

In [ ]:
## load the data
review_docs = 


## have only the business_id and text columns
review_docs = review_docs.filter(['business_id', 'text'], axis = 1)
review_docs.groupby('business_id')['text'].apply(lambda x: ' '.join(x)).reset_index()


In [90]:
# clean all the text
stopwrds = stopwords.words('english')
def cleaning_text(sentence):
    sentence = str(sentence)   #sentence as string
    sentence = sentence.lower()   #lowercase it all
    sentence = re.sub('[^\w\s]',' ', sentence)    # replace whitespace with single space
    sentence = re.sub('_',' ', sentence)    #substitute under with whitespace
    sentence = re.sub('\d+',' ', sentence)      #substitute numbers with whitespace
    cleaned = ' '.join([w for w in sentence.split() if not w in stopwrds])     #split sentence into list by words and remove stopwords
    cleaned = ' '.join([w for w , pos in pos_tag(cleaned.split()) if (pos == 'NN' or pos=='JJ' or pos=='JJR' or pos=='JJS' )])   #create sentece of word followed by its POS if its POS is of a certain type
    cleaned = ' '.join([w for w in cleaned.split() if not len(w)<=2 ])    #remove words less than two chars long (which means the POS tags are removed)
    cleaned = cleaned.strip()    #return list of words that are left and cleaned 
    return cleaned

  
# add utf-8 encoding
review_docs['textClean'] = review_docs.apply(lambda row: cleaning_text(row['text'].encode("utf8")), axis=1)

In [91]:
def split_sentence(sentence):
    words = re.split('\W+', sentence.lower())
    return [word for word in words if word != ""]

# MyDocs reading from a data frame
class MyDocs(object):
    def __iter__(self):
        for i in range(review_docs.shape[0]):
            yield doc2vec.TaggedDocument(words=split_sentence(review_docs.iloc[i,1]), tags=['%s' % review_docs.iloc[i,0]])

In [22]:
# Train the doc2vec model
mydocs = MyDocs()
model = doc2vec.Doc2Vec(mydocs, size = 200, window = 8, min_count = 5, workers = 4)
model.save("review.model")

In [23]:
# Test similar words  (I'm hoping this brings up "bun")
print model.most_similar(positive = ['tortilla', 'american'], negative = ['mexican'], topn = 5)

In [ ]:
#save a pickle file
import pickle

with open('models/doc2vec.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# saving docvecs
review_docvecs = model.docvecs

In [ ]:
#save pickle file
import pickle
with open('review_docvecs.pickle', 'wb') as handle:
    pickle.dump(review_docvecs, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Similarity/Recommender
## ignore for now

In [ ]:
# Calculate cosine similarity between two vecotrs 
def cossim(v1, v2): 
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2)) 

# return top_n values from a list
def top_n_index(l,n):
    return sorted(range(len(l)), key=lambda i: l[i])[-(n+1):-1] #-1 to take off the own product from the returned index list


def content_recommend(business_id, top_n, business_lookup, inputs):
    input_vec = inputs['business_id' == business_id]
    
    #compute similarity array
    sim_array = map(lambda v: cossim(input_vec, v), inputs)
    
    # recommendation's index (set to 50 to get enough to filter out later)
    recommendation_index = top_n_index(sim_array, 500)
    
    # recommendation's unique id
    recommendation_unique_id = [i+1 for i in recommendation_index]
    
    # recommendation's cossim values
    recommendation_cossim = [sim_array[i] for i in recommendation_index]
    
    top_products = zip(recommendation_unique_id, recommendation_cossim)
    
    # get the range of unique id for a given category prefered by user
    category_range = category_id_range(category_list)
    
    result = [i for i in top_products if i[0] in category_range]
    
    return result[-top_n:] 